In [1]:
from qiskit import QuantumCircuit, Aer, transpile 
import numpy as np 

In [18]:
from qiskit.circuit.library import GMS, TwoLocal
#from qiskit.visualization.library import _generate_circuit_library_visualization

import numpy as np
circuit = GMS(num_qubits=2, theta=[[0, np.pi/4, np.pi/8],
                                          [0, 0, np.pi/2],
                                          [0, np.pi/3, 0]])


circuit = GMS(num_qubits=2, theta=[[0,np.pi/3]])                                        
#_generate_circuit_library_visualization(circuit.decompose())
circuit.decompose().draw()

┌───────────┐
q_0: ┤0          ├
     │  Rxx(π/3) │
q_1: ┤1          ├
     └───────────┘

In [62]:
entangler_map_1 = [[0, 1], [2, 3]]
entangler_map_2 = [[ 1,2]]
entangler_map = [entangler_map_1,entangler_map_2]
two = TwoLocal(4, ['rx','rz'], ['cx'], entangler_map_1, reps=1, insert_barriers=True)
qc = QuantumCircuit(4)
qc &= two
print(qc.decompose().draw())
circ = QuantumCircuit(4)
circ.rz(np.pi/4,range(4))
print(circ)
qc.compose(circ)
print(qc)
print(qc.decompose().draw())

     ┌──────────┐┌──────────┐ ░       ░  ┌──────────┐┌───────────┐
q_0: ┤ Rx(θ[0]) ├┤ Rz(θ[4]) ├─░───■───░──┤ Rx(θ[8]) ├┤ Rz(θ[12]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐ ░  ├──────────┤├───────────┤
q_1: ┤ Rx(θ[1]) ├┤ Rz(θ[5]) ├─░─┤ X ├─░──┤ Rx(θ[9]) ├┤ Rz(θ[13]) ├
     ├──────────┤├──────────┤ ░ └───┘ ░ ┌┴──────────┤├───────────┤
q_2: ┤ Rx(θ[2]) ├┤ Rz(θ[6]) ├─░───■───░─┤ Rx(θ[10]) ├┤ Rz(θ[14]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐ ░ ├───────────┤├───────────┤
q_3: ┤ Rx(θ[3]) ├┤ Rz(θ[7]) ├─░─┤ X ├─░─┤ Rx(θ[11]) ├┤ Rz(θ[15]) ├
     └──────────┘└──────────┘ ░ └───┘ ░ └───────────┘└───────────┘
     ┌─────────┐
q_0: ┤ Rz(π/4) ├
     ├─────────┤
q_1: ┤ Rz(π/4) ├
     ├─────────┤
q_2: ┤ Rz(π/4) ├
     ├─────────┤
q_3: ┤ Rz(π/4) ├
     └─────────┘
     »
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                  

In [93]:
print(3*qc.num_qubits)
l= [i for i in range(3*qc.num_qubits)]
l
print(l[:qc.num_qubits])
l[qc.num_qubits:len(l)-qc.num_qubits]
l[-qc.num_qubits:]

12
[0, 1, 2, 3]


[8, 9, 10, 11]

In [70]:
for i in range(qc.num_qubits):
    print(i)

0
1
2
3


In [97]:
from qiskit.circuit import Parameter

num_qubits = 3
# Create parameters for single-qubit rotations
params = [Parameter(f"theta_{i}")for i in range(2*num_qubits)]

params

[Parameter(theta_0),
 Parameter(theta_1),
 Parameter(theta_2),
 Parameter(theta_3),
 Parameter(theta_4),
 Parameter(theta_5)]

In [98]:
def bonds_without_native_connectivity(qc: QuantumCircuit):
    even_bonds = []
    odd_bonds = []

    for i in range(qc.num_qubits - 1):
        if i % 2 == 0:
            even_bonds.append((i, i + 1))
        else:
            odd_bonds.append((i, i + 1))
    return even_bonds, odd_bonds

In [112]:
qc = QuantumCircuit(4)
even_bonds, odd_bonds = bonds_without_native_connectivity(qc)
one_layer = (3*qc.num_qubits+len(even_bonds)+len(odd_bonds))
parameters = [Parameter(f'theta{i}')for i in range(2*one_layer)] 
print(parameters[0:3*qc.num_qubits])
parameters[len(parameters[:3*qc.num_qubits]):one_layer]

[Parameter(theta0), Parameter(theta1), Parameter(theta2), Parameter(theta3), Parameter(theta4), Parameter(theta5), Parameter(theta6), Parameter(theta7), Parameter(theta8), Parameter(theta9), Parameter(theta10), Parameter(theta11)]


[Parameter(theta12), Parameter(theta13), Parameter(theta14)]

In [116]:
ll = [1,2,3,4,5,6]

for i in range(3):
    print(ll[i*2:(i+1)*2])

[1, 2]
[3, 4]
[5, 6]


## Mapping variables into pauli string encoding

lets say I have 
num_variables = m = number of Pauli strings  and 
k =2 chosen 
encoding is 3*nCk
for example 9 variables are encoded in 9 pauli strings 
x1 --> IXX (210)
x2 --> XXI
x3 --> XIX
x4 --> IYY
.
.
.
so on
1. so take the array of clauses in dense form and map it encoding 
2. one can think of efficient hardware mapping considering the pauli encoding is not local, i.e., two variables connected in clause may not be n.n. on the deviceS

In [1]:
from itertools import permutations

def unique_permutations(s):
    # Generate all permutations of the string
    perm_set = set(permutations(s))
    
    # Convert each tuple back to a string
    perm_list = [''.join(p) for p in perm_set]
    
    # Sort the list if you want them in a specific order
    perm_list.sort()
    return perm_list

# Test the function
string = "IIXX"
result = unique_permutations(string)
print("Unique permutations of '{}':".format(string))
for perm in result:
    print(perm)


Unique permutations of 'IIXX':
IIXX
IXIX
IXXI
XIIX
XIXI
XXII


In [2]:
def generate_Pauli_correlation_encoding(num_variables = 18,num_encoded_qubits =4, k=2):
    l = []
    dic = {}
    for pauli in ['X','Y','Z']:
        encoded_pauli_strings = unique_permutations("I"*(num_encoded_qubits-k)+k*pauli)
        l+=encoded_pauli_strings
    
    for i,j in zip(range(num_variables),l):
        dic[i] = j
        
    return l,dic
        
_,dic = generate_Pauli_correlation_encoding(num_variables=6,num_encoded_qubits=3, k=2)
dic

{0: 'IXX', 1: 'XIX', 2: 'XXI', 3: 'IYY', 4: 'YIY', 5: 'YYI'}

In [3]:
#counts = {'0011':10}
#total_shots = 10
def pauli_strings_meas(pauli_meas: str, pauli_str:str, counts: dict):
    
    idx_meas_bits = [len(pauli_str)-1-i for i,c in enumerate(pauli_str) if c==pauli_meas]
    print(idx_meas_bits)
    pauli_str_expt_val = 1
    for meas_bit, meas_count in counts.items():
        for i in idx_meas_bits:
            pauli_str_expt_val*=(-1)**int(meas_bit[len(pauli_str)-1- i])
            print(pauli_str_expt_val)
        pauli_str_expt_val = pauli_str_expt_val*meas_count
    
    return pauli_str_expt_val/total_shots
pauli_strings_meas(pauli_meas= 'X',pauli_str='IXXX',counts=counts)

[2, 1, 0]
1
-1
1


1.0

In [4]:
#counts = {'0011': 10,'1001':20}
#total_shots = 30

def pauli_strings_meas(pauli_meas: str, pauli_str: str, counts: dict) -> float:
    # Find indices of the pauli_meas in pauli_str in reverse order
    idx_meas_bits = [len(pauli_str) - 1 - i for i, c in enumerate(pauli_str) if c == pauli_meas]
    #print("Measurement bit indices:", idx_meas_bits)
    
    # Initialize expectation value accumulator
    pauli_str_expt_val = 0

    # Loop over each measured bitstring and count
    for meas_bit, meas_count in counts.items():
        # Calculate the contribution of this measurement bitstring
        bitstring_val = 1
        for i in idx_meas_bits:
            bitstring_val *= (-1) ** int(meas_bit[len(pauli_str) - 1 - i])
        pauli_str_expt_val += bitstring_val * meas_count
        # print(f"Intermediate expectation value for {meas_bit}: {bitstring_val * meas_count}")

    # Normalize by the total number of shots
    return pauli_str_expt_val /shots

# Example usage
# result = pauli_strings_meas(pauli_meas='X', pauli_str='IXXX', counts=counts)
# print("Final expectation value:", result)


In [ ]:
# Collect the signs of the expectation as the variables using var-> string mapping

# make the loss function first and set up the VQA program for parameter optimization

In [20]:
import sys 
sys.path.append('/Users/prachisharma/Dropbox/Github_projects/project-optimization/src')
sys.path.append('/Users/prachisharma/Dropbox/Github_projects/project-optimization/src/Qencoding')
from utils import *
from pauli_string_encoding import *
xor_prob =   (np.array([[2, 4, 5],
       [0, 1, 4],
       [0, 3, 4],
       [0, 1, 3],
       [1, 2, 3],
       [0, 1, 5],
      [0, 2, 5],[0, 1, 2]]),
 np.array([1, 1, 0, 0, 1, 1, 1, 1]))
#counts = {'011': 10,'001':20}
total_shots = 30
xorsat_hamiltonian = mapping_XORSAT_to_Hamiltonian(xor_prob)
print(xorsat_hamiltonian)
print(dic)

initial parameters:  [2.09503276 1.09907643 2.21382433 2.39091459 1.68819553 2.24041624
 0.62847063 2.90611931 1.6414933  1.07409113 0.83156127 0.6940304
 0.37586851 1.73841044 2.74458488 2.63118514 0.82251434 2.13986763
 0.50054394]
     ┌───────────┐  ┌────────────┐                          ┌────────────┐»
q_0: ┤ Rx(2.095) ├──┤ Rz(1.6882) ├──■───────────────────■───┤ Rx(1.6415) ├»
     ├───────────┴┐ ├────────────┤┌─┴─┐┌─────────────┐┌─┴─┐ ├────────────┤»
q_1: ┤ Rx(1.0991) ├─┤ Rz(2.2404) ├┤ X ├┤ Rz(0.82251) ├┤ X ├─┤ Rx(1.0741) ├»
     ├────────────┤┌┴────────────┤└───┘└─────────────┘└───┘┌┴────────────┤»
q_2: ┤ Rx(2.2138) ├┤ Rz(0.62847) ├──■───────────────────■──┤ Rx(0.83156) ├»
     ├────────────┤└┬────────────┤┌─┴─┐ ┌────────────┐┌─┴─┐├─────────────┤»
q_3: ┤ Rx(2.3909) ├─┤ Rz(2.9061) ├┤ X ├─┤ Rz(2.1399) ├┤ X ├┤ Rx(0.69403) ├»
     └────────────┘ └────────────┘└───┘ └────────────┘└───┘└─────────────┘»
«     ┌─────────────┐                          ░ 
«q_0: ┤ Rz(0.37587) ├───────────

In [22]:
def energy_pauli_str_encoding(xorsat_hamiltonian: Dict[str, int], pauli_str_var_encoding: Dict[int,str], counts={}):
    alpha = 1
    clause_encoding = []
    for clause_key, _ in xorsat_hamiltonian.items():
        clause_encoding.append([pauli_str_var_encoding[int(i)] for i in clause_key])
    total_energy =0
    for encoded_clause in clause_encoding:
        energy_encoded_clause = 1
        for clause in encoded_clause:
            if 'X' in clause:
                print(np.tanh(pauli_strings_meas(pauli_meas='X',pauli_str=clause,counts=counts)))
                energy_encoded_clause*=np.tanh(alpha*pauli_strings_meas(pauli_meas='X',pauli_str=clause,counts=counts))#counts_X))
            elif 'Y' in clause:
                energy_encoded_clause*=np.tanh(alpha*pauli_strings_meas(pauli_meas='Y',pauli_str=clause,counts=counts))
            else:
                energy_encoded_clause*=np.tanh(alpha*pauli_strings_meas(pauli_meas='Z',pauli_str=clause,counts=counts))
        total_energy+=energy_encoded_clause
    
    return total_energy


# energy_pauli_str_encoding(xorsat_hamiltonian=xorsat_hamiltonian, pauli_str_var_encoding = dic, counts=counts)

In [ ]:
# add optimization fucntion to the above cost function 
# reverse mapping from the final pauli strings to variables 

from qiskit.providers.fake_provider import FakeNairobiV2
from qiskit_aer import StatevectorSimulator

from typing import Dict, List, Tuple, Callable
from scipy.optimize import minimize

def get_black_box_objective_with_cost(
    num_variables: int,
    xorsat_hamiltonian: Dict[str, float],
    cost_function: Callable[[Dict[str, int]], float],
    backend=FakeNairobiV2(), 
    shots: int = 1024,  # Default number of shots for circuit execution
) -> Callable[[List[float]], float]:

    def objective_function(theta: List[float]) -> float:

        qc = brickwall_circuit(k = 2, num_variables= 16,parameters = theta)
        #qc.measure_all()
        # Execute the circuit on the specified backend
        counts = backend.run(transpile(qc), shots=shots).result().get_counts()

        # Compute the energy using the specified cost function
        energy = energy_pauli_str_encoding(xorsat_hamiltonian = xorsat_hamiltonian,pauli_str_var_encoding = dic, counts=counts)
        print("Computed Energy:", energy)  # Optional logging for debugging

        # to plot the cost function with number of iterations 
        # obj_cost.append(energy)
        return energy

    return objective_function

shots = 1024
backend = FakeNairobiV2()
sv_backend = StatevectorSimulator() 
num_variables = 6
cost_function = energy_pauli_str_encoding
initial_params = np.random.uniform(0,np.pi,(4*4+2+1))
obj_function = get_black_box_objective_with_cost(num_variables, xorsat_hamiltonian, cost_function, backend, shots)
    # Run the optimizer (COBYLA method : can be done using others)
res_sample = minimize(obj_function, initial_params, method='COBYLA', options={'maxiter':5000,'disp': True})

Need to do statevector simulator calculations since the circuit in this are smaller and can 